In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


用import input_data加载mnist数据集，加载mnist数据集报错，就使用import tensor_mnist.mnist as input_data加载mnist数据集

In [2]:
import tensor_mnist.mnist as input_data

In [3]:
mnist = input_data.read_data_sets('data/', one_hot=True)
trainimg   = mnist.train.images
trainlabel = mnist.train.labels
testimg    = mnist.test.images
testlabel  = mnist.test.labels
print ("MNIST ready")

filepath:data/train-images-idx3-ubyte.gz
Extracting data/train-images-idx3-ubyte.gz
filepath:data/train-labels-idx1-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
filepath:data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
filepath:data/t10k-labels-idx1-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST ready


In [4]:
print (trainimg.shape)
print (trainlabel.shape)
print (testimg.shape)
print (testlabel.shape)
#print (trainimg)
print (trainlabel[0])

(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [5]:
x = tf.placeholder("float", [None, 784]) 
y = tf.placeholder("float", [None, 10])  # None is for infinite 
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
# 逻辑回归建模
actv = tf.nn.softmax(tf.matmul(x, W) + b) 
# 定义损失函数，softmax交叉熵loss函数，-sum(y*logp)
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(actv), reduction_indices=1)) 
# 优化
learning_rate = 0.01
optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [6]:
# 返回最大值的索引，即为概率最大的值对应的索引（索引和标签对应）
# 和真实值对比，返回true和false
pred = tf.equal(tf.argmax(actv, 1), tf.argmax(y, 1))   
# 求准确率
accr = tf.reduce_mean(tf.cast(pred, "float")) # cast方法：true和false转为0，1
# INITIALIZER所有变量初始化
init = tf.global_variables_initializer()

小实验：tf rank eval shape argmax 方法的使用

In [8]:
sess = tf.InteractiveSession()

arr = np.array([[31, 23,  4, 24, 27, 34],
                [18,  3, 25,  0,  6, 35],
                [28, 14, 33, 22, 20,  8],
                [13, 30, 21, 19,  7,  9],
                [16,  1, 26, 32,  2, 29],
                [17, 12,  5, 11, 10, 15]])
print(tf.rank(arr).eval()) # 要打印需加eval,rank查看维度
print(tf.shape(arr).eval()) # 查看形状
print(tf.argmax(arr, axis=0).eval()) # 返回最大值的索引，0代表跨行操作
# 0 -> 31 (arr[0, 0])
# 3 -> 30 (arr[3, 1])
# 2 -> 33 (arr[2, 2])
#tf.argmax(arr, 1).eval() # 返回最大值的索引，1代表跨列运算
# 5 -> 34 (arr[0, 5])
# 5 -> 35 (arr[1, 5])
# 2 -> 33 (arr[2, 2])

2
[6 6]
[0 3 2 4 0 1]


C:\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [7]:
training_epochs = 50 # 把所有样本迭代50次
batch_size      = 100
display_step    = 5
# SESSION
sess = tf.Session()
sess.run(init) # 初始化
# 小批量的梯度下降
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(mnist.train.num_examples/batch_size)
    for i in range(num_batch): 
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys}) # 梯度下降求解
        feeds = {x: batch_xs, y: batch_ys}
        # 每一个epoch返回一个损失值
        avg_cost += sess.run(cost, feed_dict=feeds)/num_batch
    # DISPLAY
    if epoch % display_step == 0:
        feeds_train = {x: batch_xs, y: batch_ys}
        feeds_test = {x: mnist.test.images, y: mnist.test.labels}
        # 计算测试集和训练集的准确率
        train_acc = sess.run(accr, feed_dict=feeds_train)
        test_acc = sess.run(accr, feed_dict=feeds_test)
        print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
               % (epoch, training_epochs, avg_cost, train_acc, test_acc))
print ("DONE")

Epoch: 000/050 cost: 1.174406670 train_acc: 0.820 test_acc: 0.853
Epoch: 005/050 cost: 0.440944630 train_acc: 0.820 test_acc: 0.895
Epoch: 010/050 cost: 0.383359969 train_acc: 0.930 test_acc: 0.904
Epoch: 015/050 cost: 0.357285270 train_acc: 0.870 test_acc: 0.909
Epoch: 020/050 cost: 0.341516023 train_acc: 0.920 test_acc: 0.912
Epoch: 025/050 cost: 0.330553392 train_acc: 0.910 test_acc: 0.914
Epoch: 030/050 cost: 0.322364742 train_acc: 0.930 test_acc: 0.916
Epoch: 035/050 cost: 0.315963900 train_acc: 0.850 test_acc: 0.917
Epoch: 040/050 cost: 0.310717826 train_acc: 0.890 test_acc: 0.918
Epoch: 045/050 cost: 0.306355542 train_acc: 0.920 test_acc: 0.919
DONE


总结：tensorflow书写算法套路：先定义各种组件，然后运行就Ok